In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import warnings
warnings.simplefilter(action="ignore")
import time
import pandas as pd

In [3]:
#1. 웹 페이지 열기
driver = webdriver.Chrome()

url = "https://www.starbucks.co.kr/store/store_map.do"
driver.get(url)
time.sleep(2)

In [4]:
#2. 스타벅스 DT 매장만 화면에 띄우기
driver.find_element(By.CSS_SELECTOR, "#container > div > form > fieldset > div > section > article.find_store_cont > header > p > a").click()
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, "dl.opt_select_dl1>dd.right>.icon02").click()
driver.find_element(By.CSS_SELECTOR, "#storeMap > form > fieldset > div > ul > li.li2 > a").click()

#3. 전체 매장 수 확인하기
time.sleep(2)
num = driver.find_element(By.CSS_SELECTOR, "#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(2) > div.result_num_wrap.myStoreInfo > span").text
num=int(num)
num

479

In [7]:
#4. 화면에 나오지 않는 부분은 스크롤 하면서 매장정보 (매점명, 주소, 위도, 경도) 저장장
sbks=[]

from selenium.webdriver import ActionChains
for i in range(1,num+1):
    some_tag = driver.find_element(By.CSS_SELECTOR, "#mCSB_1_container > ul > li:nth-child({}) > p".format(i))
    action = ActionChains(driver)
    action.move_to_element(some_tag).perform()
    time.sleep(1)
    title = driver.find_element(By.CSS_SELECTOR, "#mCSB_1_container > ul > li:nth-child({}) > strong".format(i)).text
    address = driver.find_element(By.CSS_SELECTOR, "#mCSB_1_container > ul > li:nth-child({}) > p".format(i)).text

    data = driver.find_elements(By.CSS_SELECTOR, "#mCSB_1_container > ul > li:nth-child({})".format(i))
    lng = [lng.get_attribute("data-long") for lng in data]
    lat = [lat.get_attribute("data-lat") for lat in data]

    sbks.append({
                "name":title,
                "address":address,
                "lat":lat,
                "lng":lng
            })

In [8]:
#5. 데이터프레임으로 변환해서 보기 좋은 형태로 변환
df_sbk=pd.DataFrame(sbks)

def remove_after_newline(text):
    return text.split('\n')[0]

df_sbk['address'] = df_sbk['address'].apply(remove_after_newline)

df_sbk['lat'] = df_sbk['lat'].apply(lambda x: str(x[0])).str[:11]
df_sbk['lng'] = df_sbk['lng'].apply(lambda x: str(x[0])).str[:11]
df_sbk

,name,address,lat,lng
0,공릉DT,서울특별시 노원구 동일로 1015 (공릉동),37.6206884,127.0743116
1,태릉입구역DT,서울특별시 노원구 화랑로 469 (공릉동),37.61942,127.07932
2,구리갈매DT,경기도 구리시 갈매순환로 148 (갈매동) 1-2층,37.63237,127.1226
3,도봉로DT,서울특별시 도봉구 도봉로 635 (쌍문동),37.66118059,127.0416300
4,종암DT,서울특별시 성북구 종암로 17 (종암동),37.5919525,127.0360401
...,...,...,...,...
474,제주한담해변DT,제주특별자치도 제주시 애월읍 일주서로 6142,33.458479,126.311091
475,제주서귀포남원DT,제주특별자치도 서귀포시 남원읍 일주동로 7129,33.2828825,126.718877
476,서귀포DT,제주특별자치도 서귀포시 일주서로 11 (강정동),33.2492775,126.5065228
477,제주중문DT,제주특별자치도 서귀포시 천제연로 95(색달동),33.2549983,126.4159058


In [9]:
df_sbk.to_csv('data/스타벅스_매장정보.csv', index=False, encoding='utf-8-sig')